<h3>Toy example experiment 6.18</h3>

Use the magenpy GWASimulator based on RA's genotypes data (d1CM22_sorted.bed) to simulate phenotypes.  
the heritibility = 0.5, (Gaussian Mixture Proportion)pi = [.85, .15]

In [98]:
import numpy as np
import pandas as pd
import magenpy as mgp
import viprs as vp

In [99]:
#simulate 
g_sim = mgp.GWASimulator("CMdream1/chr22/d1CM22_sorted",
                         pi = [.85, .15],
                         h2=0.5)
g_sim.simulate()
g_sim.to_phenotype_table()

> Reading BED file...
> Generating polygenic scores...


,FID,IID,phenotype
0,0,1721000,-1.431635
1,0,1722000,-1.189151
2,0,1723000,-0.241430
3,0,1725000,0.190855
4,0,1726000,0.096196
...,...,...,...
172,0,1921000,-0.254492
173,0,1922000,-1.141154
174,0,1923000,1.829900
175,0,1924000,-1.349220


In [100]:
pd.read_csv("CMdream1/chr22/d1CM22_sorted.bim", sep="\t", header=None)


,0,1,2,3,4,5
0,22,rs4911642,-11.277499,14884399,G,A
1,22,rs2027653,-7.277181,15298335,G,A
2,22,rs5747620,-6.171966,15412698,G,A
3,22,rs5747968,-5.835598,15447504,C,A
4,22,rs2236639,-5.787480,15452483,A,G
...,...,...,...,...,...,...
7915,22,rs715586,70.074142,49510004,A,G
7916,22,rs8137951,70.076278,49512530,A,G
7917,22,rs756638,70.084762,49518559,A,G
7918,22,rs3810648,70.168330,49522492,G,A


In [101]:
g_sim.perform_gwas()
g_sim.to_summary_statistics_table().to_csv(
    "Toy_example_expr/d1CM22_sorted.sumstats", sep="\t", index=False
)

> Performing GWAS...


In [102]:
# Load summary statistics（simulate phenotype from above) and match them with perviously
gdl_sim = mgp.GWADataLoader(bed_files="CMdream1/chr22/d1CM22_sorted",
                            sumstats_files="Toy_example_expr/d1CM22_sorted.sumstats",
                            sumstats_format="magenpy")

> Reading BED file...
> Reading summary statistics file...
> Harmonizing data...


In [103]:
pd.read_csv("Toy_example_expr/d1CM22_sorted.sumstats", sep="\t", header=None)

,0,1,2,3,4,5,6,7,8,9,10
0,CHR,SNP,POS,A1,A2,MAF,N,BETA,Z,SE,PVAL
1,22,rs4911642,14884399,G,A,0.12721893491124261,169,-0.08856489538651625,-1.1513436400247115,0.07692307692307691,0.24959089035359117
2,22,rs2027653,15298335,G,A,0.3742690058479532,171,0.04977639999618155,0.6509108920698413,0.07647191129018725,0.5151040099220117
3,22,rs5747620,15412698,G,A,0.3953488372093023,172,0.06601507655785453,0.8657796124104424,0.07624928516630233,0.3866110315765857
4,22,rs5747968,15447504,C,A,0.3700564971751412,177,0.03343685249011102,0.4448483893270195,0.07516460280028288,0.6564293221677902
...,...,...,...,...,...,...,...,...,...,...,...
7916,22,rs715586,49510004,A,G,0.14204545454545456,176,0.05810125755767564,0.7708002846664442,0.07537783614444091,0.4408253185027029
7917,22,rs8137951,49512530,A,G,0.2768361581920904,177,-0.006485122129547922,-0.08627893832924659,0.07516460280028288,0.9312446808125586
7918,22,rs756638,49518559,A,G,0.2471590909090909,176,-0.08099534300311903,-1.0745246500299335,0.07537783614444091,0.28258760286066265
7919,22,rs3810648,49522492,G,A,0.048295454545454544,176,-0.10198563588087782,-1.3529923528907142,0.07537783614444091,0.1760580734042253


In [104]:
gdl_sim.compute_ld(estimator='sample',
                   output_dir='Toy_example_expr/out/ld/')

> Computing LD matrix...


Use Viprs, initialize the viprs model based on the <code> gdl_sim model</code> containing the GWAS data and the LD matrix (using xarray method) for the chr22.

In [105]:
v = vp.VIPRS(gdl_sim)

In [106]:
v.fit()

> Initializing model parameters
> Loading LD matrices into memory...
> Performing model fit...
> Using up to 1 threads.


  9%|▉         | 91/1000 [00:12<02:01,  7.50it/s]

Converged at iteration 92 || ELBO: -251.181433
> Final ELBO: -251.181433
> Estimated heritability: 0.050218
> Estimated proportion of causal variants: 0.015096


In [107]:
# Once the model converges, we can generate PRS estimates 
prs_chr22 = v.predict()
print(len(prs_chr22))
prs_chr22

> Generating polygenic scores...
177


array([-0.04695673, -0.04185944, -0.0052989 ,  0.00449479,  0.00407263,
        0.02464838, -0.01514396, -0.00339302, -0.00318481, -0.00699515,
       -0.01998505, -0.02813454,  0.05392086, -0.00299689, -0.02087477,
       -0.02607712, -0.02671616, -0.00085843, -0.05155679,  0.02365823,
        0.02751247,  0.00518783, -0.02876384,  0.01920401, -0.00412389,
       -0.04727499, -0.02427271,  0.00161893, -0.00186504, -0.01437977,
        0.00758648, -0.01168766, -0.06093111, -0.01042816, -0.01988319,
       -0.00726837, -0.02012914, -0.04258221,  0.05801946,  0.063678  ,
       -0.02080692, -0.07065526,  0.01396433, -0.00506005, -0.04390519,
        0.0291715 , -0.02503842, -0.01522802, -0.01644112,  0.02797354,
       -0.03858466, -0.01110159, -0.01030724,  0.00684037,  0.05925087,
        0.04945673, -0.01600061,  0.02109528,  0.0634981 , -0.00863217,
        0.01152387,  0.02226593, -0.00377741, -0.00109598, -0.01427131,
        0.02545948, -0.02000596, -0.0446569 ,  0.01242695,  0.00

In [108]:
# To examine posterior estimates for the model parameters, you can simply invoke the .to_table() method:
v.to_table()

,CHR,SNP,A1,A2,BETA,PIP,VAR_BETA
0,22,rs4911642,G,A,-0.000090,0.015294,0.000006
1,22,rs2027653,G,A,0.000045,0.014796,0.000006
2,22,rs5747620,G,A,0.000063,0.014952,0.000006
3,22,rs5747968,C,A,0.000033,0.014691,0.000006
4,22,rs2236639,A,G,0.000039,0.014737,0.000006
...,...,...,...,...,...,...,...
7915,22,rs715586,A,G,0.000055,0.014869,0.000006
7916,22,rs8137951,A,G,-0.000007,0.014601,0.000005
7917,22,rs756638,A,G,-0.000080,0.015143,0.000006
7918,22,rs3810648,G,A,-0.000104,0.015495,0.000006


<code>Validation</code> use another the simulate phenotype still on the d1chr22_sorted.bed as the validation part. 

In [109]:
#simulate 
g2_sim = mgp.GWASimulator("CMdream1/chr22/d1CM22_sorted",
                         pi = [.85, .15],
                         h2=0.5)
g2_sim.simulate()
a = g2_sim.to_phenotype_table()
a.to_csv("Toy_example_expr/phenotype/g2_sim_pheno.txt",sep='\t', index=False, header=False)
g2_sim.perform_gwas()
g2_sim.to_summary_statistics_table().to_csv(
    "Toy_example_expr/d1CM22_sorted_valid.sumstats", sep="\t", index=False
)

# Load summary statistics（simulate phenotype from above) and match them with perviously
gdl2_sim = mgp.GWADataLoader(bed_files="CMdream1/chr22/d1CM22_sorted",
                            phenotype_file="Toy_example_expr/phenotype/g2_sim_pheno.txt",
                            sumstats_files="Toy_example_expr/d1CM22_sorted_valid.sumstats",
                            sumstats_format="magenpy")
gdl2_sim.compute_ld(estimator='sample',
                   output_dir='Toy_example_expr/out/ld_val/')


> Reading BED file...
> Generating polygenic scores...
> Performing GWAS...


> Reading BED file...
> Reading phenotype file...
> Reading summary statistics file...
> Harmonizing data...
> Computing LD matrix...


Exception ignored in: <finalize object at 0x7fd075efd040; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 572, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/usr/lib/python3.7/tempfile.py", line 934, in _cleanup
    _rmtree(name)
  File "/usr/lib/python3.7/shutil.py", line 485, in rmtree
    onerror(os.lstat, path, sys.exc_info())
  File "/usr/lib/python3.7/shutil.py", line 483, in rmtree
    orig_st = os.lstat(path)
FileNotFoundError: [Errno 2] No such file or directory: 'temp/ld_m8ksiz1k'


In [110]:
val_prs = v.predict(gdl2_sim)

> Generating polygenic scores...


In [111]:
from viprs.eval.metrics import r2 
r2(val_prs, gdl2_sim.sample_table.phenotype)

0.0018799913512445427

Validation use the chr22 genotypes in <code> dream2 </code> file sets.  
Do the same steps as before to got <code>"CMdream2/chr22/d2CM22_sorted"</code> binary file sets (.bed + .fam + .bim).  
Sorted the SNPs from dream2 for chromosome 22 and add CM values to it. 

In [112]:
d2_bim = pd.read_csv("data/RA_challenge_qced_genotypes/dream2.bim", sep="\t", header=None)
d2_filter_bim = d2_bim[d2_bim[0] ==22][1]
d2_sorted_chr22 = d2_filter_bim.sort_values()
d2_sorted_chr22.to_csv("CMdream2/d2_sorted_chr22_bim.txt", header=False,
                       index=False, sep="\t")

In [113]:
# Use simulate method to create phenotpyes 
#simulate 
g3_sim = mgp.GWASimulator("CMdream2/chr22/d2CM22_sorted",
                         pi = [.85, .15],
                         h2=0.5)
g3_sim.simulate()
a = g3_sim.to_phenotype_table()
a.to_csv("Toy_example_expr/phenotype/val_d2chr22_pheno.txt",sep='\t', index=False, header=False)
g3_sim.perform_gwas()
g3_sim.to_summary_statistics_table().to_csv(
    "Toy_example_expr/val_d2chr22_valid.sumstats", sep="\t", index=False
)

# Load summary statistics（simulate phenotype from above) and match them with perviously
gdl3_sim = mgp.GWADataLoader(bed_files="CMdream2/chr22/d2CM22_sorted",
                            phenotype_file="Toy_example_expr/phenotype/val_d2chr22_pheno.txt",
                            sumstats_files="Toy_example_expr/val_d2chr22_valid.sumstats",
                            sumstats_format="magenpy")
gdl3_sim.compute_ld(estimator='sample',
                   output_dir='Toy_example_expr/d2chr22_out/ld_val/')

> Reading BED file...
> Generating polygenic scores...
> Performing GWAS...
> Reading BED file...
> Reading phenotype file...
> Reading summary statistics file...
> Harmonizing data...
> Computing LD matrix...


Exception ignored in: <finalize object at 0x7fd0a8337af0; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 572, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/usr/lib/python3.7/tempfile.py", line 934, in _cleanup
    _rmtree(name)
  File "/usr/lib/python3.7/shutil.py", line 485, in rmtree
    onerror(os.lstat, path, sys.exc_info())
  File "/usr/lib/python3.7/shutil.py", line 483, in rmtree
    orig_st = os.lstat(path)
FileNotFoundError: [Errno 2] No such file or directory: 'temp/ld_w2vg4k6u'


In [114]:
val2_prs = v.predict(gdl3_sim)

> Generating polygenic scores...


In [115]:
#To evaluate the accuracy of the polygenic score, 
# you can use some of the provided metrics with the viprs package. 
# For quantitative traits, it is customary to report the prediction R^2, 
# or the proportion of variance explained by the PRS:
r2(val2_prs, gdl3_sim.sample_table.phenotype)

0.03575236838564991